**Set environment**

In [1]:
suppressMessages(suppressWarnings(source("../run_config_project_sing.R")))
show_env()

You are working on        Singularity: singularity_proj_encode_fcc 
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei 
REPO DIRECTORY (FD_REPO): /data/reddylab/Kuei/repo 
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/work 
DATA DIRECTORY (FD_DATA): /data/reddylab/Kuei/data 

You are working with      ENCODE FCC 
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC 
PROJECT RESULTS (FD_RES): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results 
PROJECT SCRIPTS (FD_EXE): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/scripts 
PROJECT DATA    (FD_DAT): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/data 
PROJECT NOTE    (FD_NBK): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/notebooks 
PROJECT DOCS    (FD_DOC): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/docs 
PROJECT LOG     (FD_LOG): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/log 
PROJECT REF     (FD_REF): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/references 



## Prepare

**Set global variable**

In [2]:
TXT_FNAME_INP = "matrix.annotation.fcc_starrmpra_junke.tsv"
TXT_FNAME_OUT = "region.annotation.genome_tss.tsv"

**View files**

In [3]:
### get annotated region folder
txt_fdiry = file.path(FD_RES, "region_annotation")
vec = dir(txt_fdiry)

### assign and show
vec_txt_folder = vec
for(txt in vec){cat(txt, "\n")}

fcc_astarr_macs_input_overlap 
fcc_astarr_macs_input_union 


In [4]:
txt_fdiry = file.path(FD_RES, "region_annotation", "*", "summary")
txt_fname = TXT_FNAME_INP
txt_fglob = file.path(txt_fdiry, txt_fname)

vec = Sys.glob(txt_fglob)
for(txt in vec){cat(txt, "\n")}

/data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results/region_annotation/fcc_astarr_macs_input_overlap/summary/matrix.annotation.fcc_starrmpra_junke.tsv 
/data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results/region_annotation/fcc_astarr_macs_input_union/summary/matrix.annotation.fcc_starrmpra_junke.tsv 


## Import data

In [5]:
### loop to import data
lst = lapply(vec_txt_folder, function(txt_folder){
    ### set file directory
    txt_fdiry = file.path(FD_RES, "region_annotation", txt_folder, "summary")
    txt_fname = TXT_FNAME_INP
    txt_fpath = file.path(txt_fdiry, txt_fname)

    ### read table
    dat = read_tsv(txt_fpath, show_col_types = FALSE)
    return(dat)
})
names(lst) = vec_txt_folder

### assign and show
lst_dat_region_annot_import = lst

res = lapply(lst, dim)
print(res)

dat = lst[[1]]
fun_display_table(head(dat, 3))

$fcc_astarr_macs_input_overlap
[1] 99749    12

$fcc_astarr_macs_input_union
[1] 135016     12



Chrom,ChromStart,ChromEnd,Region,ASTARR_A,ASTARR_R,LMPRA_A,LMPRA_R,TMPRA_A,TMPRA_R,WSTARR_A,WSTARR_R
chr1,10038,10405,chr1:10038-10405,0,1,0,0,0,0,0,0
chr1,16025,16338,chr1:16025-16338,0,1,0,0,0,0,0,0
chr1,17288,17689,chr1:17288-17689,0,1,0,0,0,0,0,0


## Summarize

In [6]:
### helper function
fun = function(mat){
    vec = apply(mat, 1, sum)
    dat = tibble(
        Region    = names(vec),
        Num_Assay = vec
    )
    return(dat)
}

### init
lst = lst_dat_region_annot_import
vec = c("Chrom", "ChromStart", "ChromEnd", "Region", "Direction", "Num_Assay")

###
lst = lapply(lst, function(dat){
    ### get enhance labels
    dat_region_enhnace = dat %>% dplyr::select(Region, ends_with("_A")) 
    
    ### get repress labels
    dat_region_repress = dat %>% dplyr::select(Region, ends_with("_R"))
    
    ### combine into list
    tmp = list(
        "Active"     = dat_region_enhnace,
        "Repressive" = dat_region_repress
    )
    
    ### loop to get assay count
    tmp = lapply(tmp, function(dat){
        dat = dat %>% tibble::column_to_rownames(var = "Region")
        dat = fun(dat)
        return(dat)
    })
    
    ### arrange count table
    dat = bind_rows(tmp, .id = "Direction")
    dat = dat %>% 
        dplyr::filter(Num_Assay > 0) %>%
        tidyr::separate(
            Region, 
            into = c("Chrom", "ChromStart", "ChromEnd"), 
            remove = FALSE) %>%
        dplyr::select(!!!vec)
    return(dat)
})

### assign and show
lst_dat_region_annot_result = lst

res = lapply(lst, dim)
print(res)

dat = lst[[1]]
fun_display_table(head(dat, 3))

$fcc_astarr_macs_input_overlap
[1] 122228      6

$fcc_astarr_macs_input_union
[1] 162819      6



Chrom,ChromStart,ChromEnd,Region,Direction,Num_Assay
chr1,115429,115969,chr1:115429-115969,Active,1
chr1,184091,184563,chr1:184091-184563,Active,1
chr1,605104,605675,chr1:605104-605675,Active,1


**Explore: Count**

In [7]:
lst = lst_dat_region_annot_result
lst = lapply(lst, function(dat){
    dat = dat %>% dplyr::mutate(Note = paste0("N", Num_Assay))
    res = table(dat$Direction, dat$Note, dnn=c("Direction", "Count"))
    dat = as.data.frame(res)
    return(dat)
})

dat = bind_rows(lst, .id = "Region")
dat = dat %>% tidyr::spread(Count, Freq) %>% dplyr::mutate(Total = N1 + N2 + N3 + N4)
fun_display_table(dat)

Region,Direction,N1,N2,N3,N4,Total
fcc_astarr_macs_input_overlap,Active,27403,13617,6609,57,47686
fcc_astarr_macs_input_overlap,Repressive,71923,2610,9,0,74542
fcc_astarr_macs_input_union,Active,34986,15104,6889,60,57039
fcc_astarr_macs_input_union,Repressive,101607,4162,11,0,105780


## Save results

In [8]:
lst = lst_dat_region_annot_result
for (idx in names(lst)){

    ### get each table
    dat_region_annot_result = lst_dat_region_annot_result[[idx]]
    
    
    ### set file directory
    txt_folder = idx
    txt_fdiry = file.path(
        FD_RES, 
        "region_annotation", 
        txt_folder,
        "summary"
    )
    
    ### set file path
    txt_fname = "region.annotation.fcc_starrmpra_junke.assayvote.tsv"
    #txt_fname = "region.summary.fcc_starrmpra_junke.assayvote.tsv"
    txt_fpath = file.path(txt_fdiry, txt_fname)

    ### write table
    dat = dat_region_annot_result
    write_tsv(dat, txt_fpath)
}